In [56]:
import torch
import transformers
import os
import math
print(transformers.__version__)
import torch
model_checkpoint = "bert-base-cased"
batch_size = 16
from transformers import AutoConfig,AutoModel
import transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding,\
                                        TrainingArguments, Trainer,default_data_collator,AdamW
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)
from torch.utils.data import DataLoader,RandomSampler
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.optim.lr_scheduler import ExponentialLR
from transformers.models.bert.modeling_bert import BertPooler
from datasets import load_dataset,load_metric
import numpy as np
import random
import datasets
f1_metric = load_metric("f1")
pr_metric = load_metric('precision')
re_metric = load_metric('recall')

4.12.3


In [57]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

In [58]:
def get_data_slice(i):
                    inputs = {
                   'input_ids':i['input_ids'].cuda(),\
                   'attention_mask':i['attention_mask'].cuda(),\
                   'labels':i['labels'].cuda(),
                       }
                    return inputs 

def preprocess_function(examples):
        result = tokenizer(examples['text'], padding="max_length", max_length=512, truncation=True)
        return result

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [59]:
class SequenceModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(model_checkpoint,output_hidden_states=False,\
                                                    add_pooling_layer=False)
        self.pooler = BertPooler(self.base_model.config)
        self._init_weights(self.pooler)
        
    def _init_weights(self, modules):
        """Initialize the weights"""
        for module in modules.modules():
                if isinstance(module, torch.nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, torch.nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)
        

    def forward(self,data):
        out = self.base_model(input_ids=data['input_ids'], \
                               attention_mask=data['attention_mask'])
        out = self.pooler(out.last_hidden_state)
        return out

In [60]:
class SentiMentDomain_Clf(torch.nn.Module):
    def __init__(self,base_model,num_labels):
        self.num_labels = num_labels
        self.dropout = torch.nn.Dropout(self.base_model.config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(self.base_model.config.hidden_size,self.num_labels)
        self._init_weights(self.classifier)
    
    
    def _init_weights(self, modules):
        """Initialize the weights"""
        for module in modules.modules():
                if isinstance(module, torch.nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, torch.nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)
    
    
    def forward(self,data):
        clf_out = self.classifier(self.dropout(data))
        return clf_out

In [61]:
seed = np.random.randint(0,1000000)
fix_all_seeds(seed)

dataset = load_dataset('csv',delimiter="\t",data_files='electronics/review_labels.csv')
dataset = datasets.concatenate_datasets([dataset['train']])
dataset_src = dataset.train_test_split(0.2,shuffle=False)

dataset = load_dataset('csv',delimiter="\t",data_files='books/review_labels.csv')
dataset = datasets.concatenate_datasets([dataset['train']])
dataset_trg = dataset.train_test_split(0.2,shuffle=False)


dataset = load_dataset('csv',delimiter="\t",data_files='electronics/multitask_books_n_electronics_data.csv')
dataset = datasets.concatenate_datasets([dataset['train']])
dataset_domain = dataset.train_test_split(0.2,shuffle=False)

Using custom data configuration default-6ce991164e62b0c8
Reusing dataset csv (/ukp-storage-1/sarkar/.cache/huggingface/datasets/csv/default-6ce991164e62b0c8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-34360c82fb13fe72
Reusing dataset csv (/ukp-storage-1/sarkar/.cache/huggingface/datasets/csv/default-34360c82fb13fe72/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-5ffc5ef371034bf6


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /ukp-storage-1/sarkar/.cache/huggingface/datasets/csv/default-5ffc5ef371034bf6/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
dataset_domain['train'][900]

{'data': "The eVGA e-GeForce 7300 GS is just what I needed to play the type of games I play......My computer had intergrated video before and that just didn't cut it....With my 288 power supply it works great and was a snap to install...Also very good service from Amazon....",
 'labels': 0}

In [68]:
len(dataset_domain['train']),len(dataset_trg['train'])

(36808, 1600)

In [1]:
dataset_src['train']

NameError: name 'dataset_src' is not defined